In [2]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
from payoff_function import PayoffFunctions as payoff

### Delta Simulation with Finite-difference Method given path-dependent options
- we define epsilon such that h is of the same scale as St

In [3]:
def delta_simulation(Nsim, S0, v, sigma, dt, T, h):
    m = int(T/dt)
    S_minus_h = [[(S0-h) for _ in range(m+1)] for _ in range(Nsim)]
    S = [[(S0) for _ in range(m+1)] for _ in range(Nsim)]
    S_plus_h = [[(S0+h) for _ in range(m+1)] for _ in range(Nsim)]
    for i in tqdm(range(Nsim)):
        Z = np.random.normal(size=m)
        for j in range(1, m+1):
            S_minus_h[i][j] = S_minus_h[i][j-1]*np.exp(v*dt+sigma*np.sqrt(dt)*Z[j-1])
            S_plus_h[i][j] = S_plus_h[i][j-1]*np.exp(v*dt+sigma*np.sqrt(dt)*Z[j-1])
            S[i][j] = S[i][j-1]*np.exp(v*dt+sigma*np.sqrt(dt)*Z[j-1])
    
    return pd.DataFrame(S_minus_h), pd.DataFrame(S), pd.DataFrame(S_plus_h)

In [4]:
stock = pd.read_csv('EURO_STOXX_50_Index.csv')
stock.columns = ['Date', 'Value']
interest = pd.read_csv('euro_area_1_year_yield_curve.csv')
interest = interest.iloc[:, [0, 2]]
interest.columns = ['Date', 'Value']

In [5]:
Nsim = 10000
epsilon = 0.001
S0 = stock[stock['Date']=='2023-08-09'].iloc[0, 1] # = 4317.33
h = epsilon*S0
r = 0.033256009999999996
v = 0.01866808783453991
sigma = 0.17080938010226537
dt = 1/252
T = 485*dt

In [7]:
S_minus_h, S, S_plus_h = delta_simulation(Nsim, S0, v, sigma, dt, T, h)

100%|██████████| 10000/10000 [00:25<00:00, 392.51it/s]


In [8]:
sim_Note = np.exp(-r*T)*S.apply(payoff.payoff_original, axis=1)
sim_Note_plus_h = np.exp(-r*T)*S_plus_h.apply(payoff.payoff_original, axis=1)
sim_Note_minus_h = np.exp(-r*T)*S_minus_h.apply(payoff.payoff_original, axis=1)

In [9]:
note_MC = np.mean(sim_Note)
note_MC_plus_h = np.mean(sim_Note_plus_h)
note_MC_minus_h = np.mean(sim_Note_minus_h)

In [11]:
delta_MC = (note_MC_plus_h-note_MC_minus_h)/2*h
print(f"Simulated Delta for 9 Aug 2023: {delta_MC}")

Simulated Delta for 9 Aug 2023: 7.22769217120873
